In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models

In [2]:
# loading the chestmnist dataset
data_path = "data"
data = np.load(f"{data_path}/chestmnist_224.npz")

train_images = data['train_images']
train_labels = data['train_labels']

validation_images = data['val_images']
validation_labels = data['val_labels']

test_images = data['test_images']
test_labels = data['test_labels']


In [3]:
print(data)

NpzFile 'data/chestmnist_224.npz' with keys: train_images, train_labels, val_images, val_labels, test_images...


In [4]:
print(train_images[0].shape)

(224, 224)


In [5]:
print(type(train_images))

<class 'numpy.ndarray'>


In [6]:
print(f"The shape of train images: {train_images.shape}, train labels: {train_labels.shape}")
print(f"The shape of validation images: {validation_images.shape}, validation labels: {validation_labels.shape}")
print(f"The shape of test images: {test_images.shape}, test labels: {test_labels.shape}")


The shape of train images: (78468, 224, 224), train labels: (78468, 14)
The shape of validation images: (11219, 224, 224), validation labels: (11219, 14)
The shape of test images: (22433, 224, 224), test labels: (22433, 14)


In [7]:
class ChestMNISTDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
    
    def __len__(self): 
        return len(self.images) # number of samples
    
    def __getitem__(self, idx):
        img = self.images[idx]

        if self.transform: # apply transformations
            img = self.transform(img)

        label = self.labels[idx]
        label = torch.tensor(label, dtype=torch.float)
        
        return img, label

In [8]:
from torchvision import transforms # to apply transformations to the images

transforms_with_augmentation = transforms.Compose([
    transforms.ToTensor(), # convert to tensor
    transforms.RandomRotation(degrees=10), # Randomly rotates the image between -10 and +10 degrees
    transforms.Normalize((0.5,), (0.5,)) # normalize the images
])

transforms = transforms.Compose([
    transforms.ToTensor(), # convert to tensor
    transforms.Normalize((0.5,), (0.5,)) # normalize the images
])

In [10]:
train_dataset = ChestMNISTDataset(train_images, train_labels, transform=transforms_with_augmentation)
validation_dataset = ChestMNISTDataset(validation_images, validation_labels, transform=transforms)
test_dataset = ChestMNISTDataset(test_images, test_labels, transform=transforms)

In [11]:
# creating the dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=1000, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [13]:
model = models.resnet18(weights=None) # not pretrained
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False) # Change input channels from 3 to 1 (grayscale images)
model.fc = nn.Linear(512, 14)  # final fully connected layer

model = model.to(device) # move the model to the GPU

In [16]:
loss = nn.BCEWithLogitsLoss()  # for multi-label classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
training_epochs = 10

In [ ]:
import time
from sklearn.metrics import roc_auc_score

train_losses = []
train_aucs = []
val_losses = []
val_aucs = []
epoch_times = []

for epoch in range(training_epochs):
    model.train() # set the model to training mode
    total_loss = 0
    all_outputs = []
    all_targets = []
    
    start_time = time.time()

    for batch_idx, (inputs, targets) in enumerate(train_loader):

        inputs, targets = inputs.to(device), targets.to(device) # move to the GPU
        optimizer.zero_grad() # zero the gradients
        outputs = model(inputs) # forward pass
        loss_value = loss(outputs, targets) # compute loss
        loss_value.backward() # backward pass
        optimizer.step() # update weights

        total_loss += loss_value.item()
        all_outputs.append(torch.sigmoid(outputs).detach().cpu())
        all_targets.append(targets.cpu())

    all_outputs = torch.cat(all_outputs).numpy()
    all_targets = torch.cat(all_targets).numpy()
    
    avg_loss = total_loss / len(train_loader) # average loss for the epoch
    auc = roc_auc_score(all_targets, all_outputs, average='macro')
    
    train_losses.append(avg_loss)
    train_aucs.append(auc)
    
    # Validation
    model.eval()
    val_total_loss = 0
    val_outputs = []
    val_targets = []
    
    with torch.no_grad():
        for inputs, targets in validation_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            val_loss_value = loss(outputs, targets)
            val_total_loss += val_loss_value.item()
            val_outputs.append(torch.sigmoid(outputs).cpu())
            val_targets.append(targets.cpu())
    
    val_outputs = torch.cat(val_outputs).numpy()
    val_targets = torch.cat(val_targets).numpy()
    
    val_avg_loss = val_total_loss / len(validation_loader)
    val_auc = roc_auc_score(val_targets, val_outputs, average='macro')
    
    val_losses.append(val_avg_loss)
    val_aucs.append(val_auc)
    
    epoch_time = time.time() - start_time
    epoch_times.append(epoch_time)
    
    print(f"Epoch [{epoch+1}/{training_epochs}], Train Loss: {avg_loss:.4f}, Train AUC: {auc:.4f}, Val Loss: {val_avg_loss:.4f}, Val AUC: {val_auc:.4f}, Time: {epoch_time:.2f}s")

Epoch [1/2], Avg Loss: 0.1751
Epoch [2/2], Avg Loss: 0.1655


In [19]:
# calculate training accuracy

model.eval() # set the model to evaluation mode
with torch.no_grad():
    all_predictions = []
    all_labels = []
    
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        predictions = torch.sigmoid(outputs) > 0.5
        
        all_predictions.append(predictions.cpu())
        all_labels.append(targets.cpu())
    
    all_predictions = torch.cat(all_predictions)
    all_labels = torch.cat(all_labels)
    
    train_accuracy = (all_predictions == all_labels).float().mean().item()
    print(f"Train Accuracy: {train_accuracy:.4f}")

Train Accuracy: 0.9488


In [20]:
# calculate validation accuracy

model.eval() # set the model to evaluation mode
with torch.no_grad():
    all_predictions = []
    all_labels = []
    
    for inputs, targets in validation_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        predictions = torch.sigmoid(outputs) > 0.5
        
        all_predictions.append(predictions.cpu())
        all_labels.append(targets.cpu())
    
    all_predictions = torch.cat(all_predictions)
    all_labels = torch.cat(all_labels)
    
    validataion_accuracy = (all_predictions == all_labels).float().mean().item()
    print(f"Validation Accuracy: {validataion_accuracy:.4f}")

Validation Accuracy: 0.9492


In [21]:
# calculate test accuracy

model.eval() # set the model to evaluation mode
with torch.no_grad():
    all_predictions = []
    all_labels = []
    
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        predictions = torch.sigmoid(outputs) > 0.5
        
        all_predictions.append(predictions.cpu())
        all_labels.append(targets.cpu())
    
    all_predictions = torch.cat(all_predictions)
    all_labels = torch.cat(all_labels)
    
    test_accuracy = (all_predictions == all_labels).float().mean().item()
    print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.9477
